In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist #cifar 같은거 써도 됨
#tf.enable_eager_execution()
from time import time
import os
print(tf.__version__)

def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

2.3.0


In [2]:
def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1] : batch_size, height, width, channel
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data) #0~255 -> 0~1

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10] #one hot incoding
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels


In [3]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(channel, weight_init) :
    return tf.keras.layers.Dense(units = channel, use_bias = True, kernel_initializer=weight_init)

def sigmoid() :
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [4]:
# Class 

# class create_model_class(tf.keras.Model):
#     def __init__(self, label_dim):
#         super(create_model_class, self).__init__()
#         weight_init = tf.keras.initializers.glorot_uniform()

#         self.model = tf.keras.Sequential()
#         self.model.add(flatten())

#         for i in range(2):
#             self.model.add(dense(256, weight_init))
#             self.model.add(relu())

#         self.model.add(dense(label_dim, weight_init))

#     def call(self, x, training=None, mask=None):
#         x = self.model(x)
#         return x

#Function

def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        #model.add(sigmoid())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [5]:
def loss_fn (model, images, labels) : 
    logits = model(images, training = True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred = logits, y_true = labels, from_logits = True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training = False)
    prediction = tf.equal(tf.argmax(labels, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [6]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))


In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [11]:

if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.3423, train_loss: 2.16572618, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    1/  468] time: 0.4302, train_loss: 2.10702419, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    2/  468] time: 0.5173, train_loss: 2.04641151, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    3/  468] time: 0.6103, train_loss: 2.03152251, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    4/  468] time: 0.6939, train_loss: 1.90503383, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    5/  468] time: 0.7752, train_loss: 1.84048271, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    6/  468] time: 0.8597, train_loss: 1.73706496, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    7/  468] time: 0.9471, train_loss: 1.58393538, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    8/  468] time: 1.0393,

Epoch: [ 0] [   74/  468] time: 6.6385, train_loss: 0.32029817, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   75/  468] time: 6.7200, train_loss: 0.38901326, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   76/  468] time: 6.8015, train_loss: 0.41693664, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   77/  468] time: 6.8835, train_loss: 0.30380565, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   78/  468] time: 6.9684, train_loss: 0.37628171, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   79/  468] time: 7.0559, train_loss: 0.30161929, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   80/  468] time: 7.1422, train_loss: 0.36595562, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   81/  468] time: 7.2266, train_loss: 0.38834122, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   82/  468] time: 7.3067, train_loss: 0.29258221, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0

Epoch: [ 0] [  151/  468] time: 13.1456, train_loss: 0.37071419, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  152/  468] time: 13.2282, train_loss: 0.31648043, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  153/  468] time: 13.3113, train_loss: 0.15916586, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  154/  468] time: 13.3949, train_loss: 0.23846450, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  155/  468] time: 13.4763, train_loss: 0.28375739, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  156/  468] time: 13.5589, train_loss: 0.24668512, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  157/  468] time: 13.6464, train_loss: 0.19995306, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  158/  468] time: 13.7291, train_loss: 0.27653536, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  159/  468] time: 13.8112, train_loss: 0.18221989, train_accuracy: 1.0000, test_Accuracy: 1.0000
E

Epoch: [ 0] [  227/  468] time: 19.8896, train_loss: 0.19068584, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  228/  468] time: 19.9771, train_loss: 0.33072630, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  229/  468] time: 20.0723, train_loss: 0.23323847, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  230/  468] time: 20.1676, train_loss: 0.15616195, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  231/  468] time: 20.2500, train_loss: 0.14587437, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  232/  468] time: 20.3420, train_loss: 0.15911071, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  233/  468] time: 20.4295, train_loss: 0.21663615, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  234/  468] time: 20.5158, train_loss: 0.17649710, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  235/  468] time: 20.6058, train_loss: 0.17030448, train_accuracy: 1.0000, test_Accuracy: 1.0000
E

Epoch: [ 0] [  303/  468] time: 26.5453, train_loss: 0.21412800, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  304/  468] time: 26.6284, train_loss: 0.14737497, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  305/  468] time: 26.7102, train_loss: 0.20887506, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  306/  468] time: 26.7917, train_loss: 0.08256812, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  307/  468] time: 26.8776, train_loss: 0.23078504, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  308/  468] time: 26.9614, train_loss: 0.16164623, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  309/  468] time: 27.0482, train_loss: 0.13521817, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  310/  468] time: 27.1326, train_loss: 0.38657105, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  311/  468] time: 27.2164, train_loss: 0.18078601, train_accuracy: 1.0000, test_Accuracy: 1.0000
E

Epoch: [ 0] [  378/  468] time: 32.9157, train_loss: 0.17027637, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  379/  468] time: 33.0075, train_loss: 0.12438503, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  380/  468] time: 33.0970, train_loss: 0.19545758, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  381/  468] time: 33.1865, train_loss: 0.16518912, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  382/  468] time: 33.2713, train_loss: 0.09365964, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  383/  468] time: 33.3583, train_loss: 0.12832057, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  384/  468] time: 33.4432, train_loss: 0.13632759, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  385/  468] time: 33.5267, train_loss: 0.04385360, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  386/  468] time: 33.6102, train_loss: 0.06521802, train_accuracy: 1.0000, test_Accuracy: 1.0000
E

Epoch: [ 0] [  452/  468] time: 39.4047, train_loss: 0.20651798, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  453/  468] time: 39.4885, train_loss: 0.08845642, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  454/  468] time: 39.5781, train_loss: 0.10849533, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  455/  468] time: 39.6684, train_loss: 0.12923785, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  456/  468] time: 39.7601, train_loss: 0.11636014, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  457/  468] time: 39.8454, train_loss: 0.09130089, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  458/  468] time: 39.9418, train_loss: 0.10731015, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  459/  468] time: 40.0435, train_loss: 0.11280535, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  460/  468] time: 40.1360, train_loss: 0.22171134, train_accuracy: 1.0000, test_Accuracy: 1.0000
E